# Load libraries

In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool
from src.tools import *

import warnings
warnings.filterwarnings("ignore")

# Read data

In [2]:
df = pd.read_csv('../data/prepared_data.csv')
df_fe = pd.read_csv('../data/data_fe.csv')

# Train Catboost

## Test data without upsampling

### Find feature importance

In [3]:
CatBoost = CatBoostClassifier(verbose=False,random_seed=42)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df_fe['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])

train_pool = Pool(data=X_train, label=y_train)

In [5]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test )

{'auc_score': 0.9003741709983868, 'f1_score': 0.9241142395957508}

In [6]:
feature_importance.to_csv('../data/feature_imporstance_WS.csv', index = False)

### Find best amount of features

In [7]:
search_num_features(df_fe,feature_importance, step = 5)

  0%|          | 0/25 [00:00<?, ?it/s]

F1_score - 0.6734304006608839, num_features - 0, AUC_score = 0.6151303997132103


  4%|▍         | 1/25 [00:03<01:17,  3.21s/it]

F1_score - 0.7779974810761374, num_features - 1, AUC_score = 0.7291629324251659


  8%|▊         | 2/25 [00:06<01:16,  3.34s/it]

...

F1_score - 0.9329922027290448, num_features - 36, AUC_score = 0.9180744757124933


 96%|█████████▌| 24/25 [03:54<00:14, 14.31s/it]

F1_score - 0.9296546462113844, num_features - 36, AUC_score = 0.9093027424269583


100%|██████████| 25/25 [04:09<00:00, 14.68s/it]


Best F1_score - 0.9345060499209441, num_features - 36


## Test data with upsampling

### Find feature importance

In [8]:
CatBoost = CatBoostClassifier(verbose=False,random_seed=42)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_fe.drop(columns = ['Machine failure']),
                                                    df_fe['Machine failure'],
                                                    test_size=0.33,
                                                    random_state=42,
                                                    stratify = df['Machine failure'])
X_train, y_train = upsampling(1,X_train, y_train)

train_pool = Pool(data=X_train, label=y_train)

In [10]:
CatBoost.fit(train_pool)

feature_importance = pd.DataFrame({'feature_importance': CatBoost.get_feature_importance(train_pool), 
              'feature_names': X_test.columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
              
check_result(CatBoost, X_test, y_test)

{'auc_score': 0.9225387614267789, 'f1_score': 0.9356730254831562}

In [11]:
feature_importance.to_csv('../data/feature_imporstance.csv',index = False)

### Find best amount of features

In [12]:
search_num_features(df_fe, feature_importance, upsamp_func = True, step = 5)

  0%|          | 0/25 [00:00<?, ?it/s]

F1_score - 0.5030190363209421, num_features - 0, AUC_score = 0.7000358487184083


  4%|▍         | 1/25 [00:08<03:32,  8.86s/it]

F1_score - 0.8971506553910363, num_features - 1, AUC_score = 0.9194019985660513


  8%|▊         | 2/25 [00:18<03:25,  8.95s/it]

...

F1_score - 0.9366441795347409, num_features - 111, AUC_score = 0.9140807044273167


 96%|█████████▌| 24/25 [06:29<00:21, 21.69s/it]

F1_score - 0.9399027517255195, num_features - 111, AUC_score = 0.9228524377128516


100%|██████████| 25/25 [06:48<00:00, 20.87s/it]


Best F1_score - 0.9399027517255195, num_features - 111
